In [ ]:
player_gamelog = playergamelog.PlayerGameLog(player_id = 2544, season = 2022)
df_games = player_gamelog.get_data_frames()
player_df = df_games[0]
player_df['FPG'] = 1* player_df['PTS'] + 0.5*player_df['FG3M'] + 1.25*player_df['REB'] + 1.5*player_df['AST'] + 2*player_df['STL'] + 2*player_df['BLK'] - 0.5*player_df['TOV']
player_df['FPM'] = player_df['FPG']/player_df['MIN']
fantasy_df = player_df[['GAME_DATE', 'MATCHUP', 'MIN', 'PTS', 'FG3M', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FPG', 'FPM']]
fantasy_df.head()


In [ ]:
def get_fantasy_df():
    """
    Read in csv files from data file and return only valuable dataset to us.
    Will be automated to account for different years
    """
    csv_path = Path("data/2023.csv")
    player_df = pd.read_csv(csv_path)
    player_df['FPG'] = 1* player_df['PTS'] + 0.5*player_df['3P'] + 1.25*player_df['TRB'] + 1.5*player_df['AST'] + 2*player_df['STL'] + 2*player_df['BLK'] -0.5*player_df['TOV'] 
    player_df['FPM'] = player_df['FPG']/player_df['MP']
    fantasy_df = player_df[['Player', 'Tm', 'Pos', 'MP', 'PTS', '3P', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'FPG', 'FPM']]
    fantasy_df = fantasy_df.drop_duplicates(subset = ['Player'], keep='last')
    
    return fantasy_df

In [ ]:
def get_lines_data():
    """
    2023 Lines data sourced from covers.com.
    Function reads and cleans csv file
    """
    csv_path = Path("data/Lines.csv")
    lines_df = pd.read_csv(csv_path)
    lines_df["AvL"] = lines_df['PPG'] + lines_df['PAG']
    lines_df = lines_df[['Tm','Team', 'AvL']]
    lines_df["Margin"] = lines_df["AvL"]/lines_df["AvL"].mean()

    return lines_df

In [ ]:
def get_minutes_data():
    """
    Returns minutes of the last 10 games per player. 
    Use this as expected minutes.
    """
    csv_path = Path("data/NBA_minutes.csv")
    minutes_df = pd.read_csv(csv_path)
    minutes_df = minutes_df.drop_duplicates(subset = ['Player'], keep='first')
    minutes_df = minutes_df[['Player', 'L10']]
    
    return minutes_df

In [ ]:
def get_projection_data():
    """
    Function reads through the 3 databases and cleans through to return 
    Player, Team, Postion and Projected Points
    """

    df = get_fantasy_df()
    line_df = get_lines_data()
    minutes_df = get_minutes_data()

    #Finding the Position and Player avg. points per game.
    group_team = df.groupby(["Tm", "Pos"])["FPG"].max()
    group_team_df = group_team.unstack(level="Pos")

    #Filling the null values with mean
    group_team_df = group_team_df.fillna(group_team_df.mean())

    OVP_df = group_team_df.copy()
    OVP_df = OVP_df/OVP_df.mean()

    clean = df.copy()
    clean = pd.merge(OVP_df, clean, on=["Tm"])
    clean["OVP"] = clean.lookup(clean.index, clean.Pos)

    clean = pd.merge(line_df, clean, on=["Tm"])

    clean = pd.merge(minutes_df, clean, on=["Player"])

    clean = clean[['Player', 'Tm', 'Pos', 'FPG', 'FPM', 'OVP', 'Margin', 'L10']]
    clean["Projected_Pts"] = clean.FPM * clean.OVP * clean.Margin * clean.L10 
    clean = clean[['Player', 'Tm', 'Pos', 'Projected_Pts']]
    
    return clean

In [ ]:
def get_lines_data():
    """
    2023 Lines data sourced from covers.com.
    Function reads and cleans csv file
    """
    csv_path = Path("data/Lines.csv")
    lines_df = pd.read_csv(csv_path)
    lines_df["AvL"] = lines_df['PPG'] + lines_df['PAG']
    lines_df = lines_df[['Tm','Team', 'AvL']]
    lines_df["Margin"] = line_df["AvL"]/line_df["AvL"].mean()

    return lines_df

In [ ]:
def get_playerlog_plot(player_name, season, logs):
    """
    Returns player plot against average over time
    """
    df = get_player_logs(player_name, season, logs)
    fig = px.line(lebron_James, x="GAME_DATE", y="FPG", title="FPG")
    fig.add_hline(y=lebron_James["FPG"].mean(), line_width=3, line_dash="dash", line_color="green")
    return HTML(fig.to_html())

In [ ]:
player_gamelog = playergamelog.PlayerGameLog(player_id = 2544, season = 2022)
df_games = player_gamelog.get_data_frames()
player_df = df_games[0]
player_df['FPG'] = 1* player_df['PTS'] + 0.5*player_df['FG3M'] + 1.25*player_df['REB'] + 1.5*player_df['AST'] + 2*player_df['STL'] + 2*player_df['BLK'] - 0.5*player_df['TOV']
player_df['FPM'] = player_df['FPG']/player_df['MIN']
fantasy_df = player_df[['GAME_DATE', 'MATCHUP', 'MIN', 'PTS', 'FG3M', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FPG', 'FPM']]
fantasy_df.head()

In [ ]:
def player_id(playername):
    player = [player for player in player_dict if player['full_name'] == playername][0]
    player_id = player['id']
    return player_id

#getting team's ID
#how to use function: player_id("Team's Name") 
#for example: team_id('Golden State Warriors')
def team_id(teamname):
    team_find = [x for x in teams if x['full_name'] == teamname][0]
    team_id_no = team_find['id']
    return team_id_no

#creating a function to get player's data from specific year
#type game_log(playerID, season's year)
#for example: game_log(2544,2018)

def game_log(playerid,season):
    player_gamelog = playergamelog.PlayerGameLog(player_id = playerid, season = season)
    df_games = player_gamelog.get_data_frames()
    return df_games[0]

#creating a function to get a player's lifetime career in basketball
#type all_season_log(playerID using earlier function)
def all_season_log(playerid):
    gamelog_bron_all = playergamelog.PlayerGameLog(player_id = playerid, season = SeasonAll.all)
    df_bron_games_all = gamelog_bron_all.get_data_frames()
    return df_bron_games_all[0]

#creating a function to get a team's stats in basketball
#type bkb_teamlog(Team's ID using earlier function)

def bkb_teamlog(teamid):
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable = teamid).get_data_frames()[0]
    return team_games


In [ ]:
#extracting raw data using nba_api package
player_dict = players.get_players()
teams = teams.get_teams()

#converting both dictionaries into Dataframes
team = pd.DataFrame(teams)
player = pd.DataFrame(player_dict)

#to check if there are any null values in the raw data
player.isnull().sum(), team.isnull().sum()